In [121]:
# タイタニック生存予測

In [122]:
# ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [123]:
%cd /content/drive/MyDrive/kaggle/titanic

/content/drive/MyDrive/kaggle/titanic


In [124]:
import pandas as pd
import numpy as np

In [125]:
# データの読み込み
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [126]:
# データの先頭の５行を表示
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [127]:
# データの型を調べる
train.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [128]:
# nullかどうかを調べてから、その数を調べる
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [129]:
test.dtypes

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [130]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [131]:
x = train[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
y = train[['Survived']]

In [132]:
# one-hotエンコーディングをする
# 展開してそれぞれ0か１を入れる
x = pd.get_dummies(x, columns=['Pclass', 'Sex','Embarked'])
x.head()

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,22.0,1,0,7.2500,0,0,1,0,1,0,0,1
1,38.0,1,0,71.2833,1,0,0,1,0,1,0,0
2,26.0,0,0,7.9250,0,0,1,1,0,0,0,1
3,35.0,1,0,53.1000,1,0,0,1,0,0,0,1
4,35.0,0,0,8.0500,0,0,1,0,1,0,0,1


In [133]:
# nullの数を調べる
x['Age'].isnull().sum()

177

In [134]:
# nullを平均で埋める
x['Age'] = x['Age'].fillna(x['Age'].mean())
# 埋まったか確認
x['Age'].isnull().sum()

0

In [135]:
# 訓練データとテストデータに分割
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)

In [136]:
# 標準化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [137]:
# ニューラルネット
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

In [138]:
model = keras.Sequential()
model.add(Dense(20, input_dim=12))
model.add(Activation('relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


In [139]:
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

In [140]:
model.fit(x_train_scaled, y_train, epochs=1000, verbose=2)

Epoch 1/1000
21/21 - 1s - loss: 0.7106 - accuracy: 0.5689 - 570ms/epoch - 27ms/step
Epoch 2/1000
21/21 - 0s - loss: 0.6413 - accuracy: 0.6497 - 35ms/epoch - 2ms/step
Epoch 3/1000
21/21 - 0s - loss: 0.5922 - accuracy: 0.6811 - 35ms/epoch - 2ms/step
Epoch 4/1000
21/21 - 0s - loss: 0.5557 - accuracy: 0.7365 - 36ms/epoch - 2ms/step
Epoch 5/1000
21/21 - 0s - loss: 0.5315 - accuracy: 0.7725 - 35ms/epoch - 2ms/step
Epoch 6/1000
21/21 - 0s - loss: 0.5113 - accuracy: 0.7919 - 39ms/epoch - 2ms/step
Epoch 7/1000
21/21 - 0s - loss: 0.4956 - accuracy: 0.7964 - 38ms/epoch - 2ms/step
Epoch 8/1000
21/21 - 0s - loss: 0.4829 - accuracy: 0.7994 - 47ms/epoch - 2ms/step
Epoch 9/1000
21/21 - 0s - loss: 0.4709 - accuracy: 0.8054 - 37ms/epoch - 2ms/step
Epoch 10/1000
21/21 - 0s - loss: 0.4603 - accuracy: 0.8084 - 36ms/epoch - 2ms/step
Epoch 11/1000
21/21 - 0s - loss: 0.4519 - accuracy: 0.8114 - 46ms/epoch - 2ms/step
Epoch 12/1000
21/21 - 0s - loss: 0.4454 - accuracy: 0.8159 - 38ms/epoch - 2ms/step
Epoch 13/10

In [141]:
# テストデータで評価
model.fit(x_test_scaled, y_test, epochs=1000, verbose=2)

Epoch 1/1000
7/7 - 0s - loss: 0.7704 - accuracy: 0.8072 - 18ms/epoch - 3ms/step
Epoch 2/1000
7/7 - 0s - loss: 0.7309 - accuracy: 0.8117 - 16ms/epoch - 2ms/step
Epoch 3/1000
7/7 - 0s - loss: 0.6709 - accuracy: 0.8161 - 18ms/epoch - 3ms/step
Epoch 4/1000
7/7 - 0s - loss: 0.6307 - accuracy: 0.8117 - 18ms/epoch - 3ms/step
Epoch 5/1000
7/7 - 0s - loss: 0.5884 - accuracy: 0.8161 - 22ms/epoch - 3ms/step
Epoch 6/1000
7/7 - 0s - loss: 0.5592 - accuracy: 0.8117 - 20ms/epoch - 3ms/step
Epoch 7/1000
7/7 - 0s - loss: 0.5216 - accuracy: 0.8117 - 16ms/epoch - 2ms/step
Epoch 8/1000
7/7 - 0s - loss: 0.4931 - accuracy: 0.8117 - 18ms/epoch - 3ms/step
Epoch 9/1000
7/7 - 0s - loss: 0.4698 - accuracy: 0.8161 - 11ms/epoch - 2ms/step
Epoch 10/1000
7/7 - 0s - loss: 0.4479 - accuracy: 0.8161 - 26ms/epoch - 4ms/step
Epoch 11/1000
7/7 - 0s - loss: 0.4314 - accuracy: 0.8161 - 19ms/epoch - 3ms/step
Epoch 12/1000
7/7 - 0s - loss: 0.4222 - accuracy: 0.8251 - 17ms/epoch - 2ms/step
Epoch 13/1000
7/7 - 0s - loss: 0.4031

In [142]:
# submit
x_test = test[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
submit = test[['PassengerId']]

x_test = pd.get_dummies(x_test, columns=['Pclass', 'Sex','Embarked'])

# Fareのnullを埋める
# nullにFareの平均値を埋め込む
x_test['Fare'] = x_test['Fare'].fillna(x_test['Fare'].mean())
x_test['Age'] = x_test['Age'].fillna(x_test['Age'].mean())
# 標準化
scaler = StandardScaler()
scaler.fit(x_test)
x_test_scaled = scaler.transform(x_test)

submit['Survived'] = model.predict(x_test_scaled)
submit.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,PassengerId,Survived
0,892,4.328449e-09
1,893,5.408505e-02
2,894,3.308938e-10
3,895,1.697540e-04
4,896,4.062998e-01


In [143]:
# nullを0で埋める
submit['Survived'].isnull().sum()

0

In [144]:
# one hot エンコーディングしたい

for s in submit['Survived']:
  if s > 0.5:
    s = 1
  elif s <= 0.5:
    s = 0

submit.head()

,PassengerId,Survived
0,892,4.328449e-09
1,893,5.408505e-02
2,894,3.308938e-10
3,895,1.697540e-04
4,896,4.062998e-01


In [145]:
# one hot エンコーディングしたい

for i in range(len(submit['Survived'])):
  if submit['Survived'][i] > 0.5:
    submit['Survived'][i] = 1
  elif submit['Survived'][i] <= 0.5:
    submit['Survived'][i] = 0

submit['Survived']
submit.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,PassengerId,Survived
0,892,0.0
1,893,0.0
2,894,0.0
3,895,0.0
4,896,0.0


In [146]:
# csvファイルを作成
submit.to_csv('submission/submit06.csv', index=False)